# Paradigma funcional en Python

## Concepto de Estado
### Paradigma imperativo
Procedimiento que se apoya en variable global para incrementar un contador.

In [ ]:
contador: int = 0

def incrementar_contador():
    global contador
    contador += 1

incrementar_contador()
print(contador)

### POO
Versión de contador utilizando una clase. El estado se mantiene encapsulado en el objeto.

In [ ]:
class Contador:
    def __init__(self, valor_inicial=0):
        self._valor: int = valor_inicial

    def actual(self) -> int:
        return self._valor
    
    def incrementar(self) -> int:
        self._valor += 1
        return self._valor
    
contador: Contador = Contador()
contador.incrementar()
print(contador.actual())

### Paradigma funcional
Versión de contador con una función pura.

In [ ]:
def incrementar_contador(contador: int) -> int:
    return contador + 1

contador = incrementar_contador(0)
print(contador)     # Salida: 1

In [ ]:
class ConjuntoInmutable:
    # Implementar
    pass


conjunto = ConjuntoInmutable('a', 1, 3, 'b', 5)
conjunto.elementos                  # ('a', 1, 3, 'b', 5)
conjunto.elementos[2] = 10          # Error
conjunto.elementos = (1, 3, 'a')    # Error

## Ciudadanos de primera clase
Función de orden superior que aplica una función a cada elemento de una lista.

In [ ]:
from typing import Callable, TypeVar, Union
from collections.abc import Sequence

Numerico = Union[int, float]

T = TypeVar("T")

def aplicar_operacion(lista: Sequence[T], operacion: Callable[[T], T]) -> Sequence[T]:
    resultado = []
    for elemento in lista:
        resultado.append(operacion(elemento))
    return resultado

# Definición de funciones que se aplicarán a la lista
def cuadrado(x: Numerico) -> Numerico:
    return x * x

def inverso(x: Numerico) -> Numerico:
    return 0 - x

# Uso de funcion de orden superior
numeros: list[int] = [1, -2, 3, -4, 5, -6]
numeros_cuadrados = aplicar_operacion(numeros, cuadrado)  # Elevar al cuadrado
numeros_inversos = aplicar_operacion(numeros, inverso)   # Inverso aditivo

print(numeros_cuadrados)  # [1, 4, 9, 16, 25, 36]
print(numeros_inversos)  # [-1, 2, -3, 4, -5, 6]

### Ejercicio: Función de orden superior
- Implementar una función llamada _wrapper_ que reciba por parámetro a otra función _f_ sin argumentos, la ejecute e imprima en pantalla el mensaje de ejecución: "Ejecutada f()".
- Extender la función _wrapper_ de forma que pueda aceptar cualquier función con argumentos variables y se puedan pasar también desde la función _wrapper_ para que se invoquen en _f_. Por ejemplo, si _f_ acepta 3 argumentos, éstos deberían también pasarse a _wrapper_ para que se invoque _f(arg1, arg2, arg3)_ dentro.

_TIP: Ver el type hint Callable del módulo [typing](https://docs.python.org/3.12/library/typing.html?highlight=typing#annotating-callable-objects)._

_TIP 2: Ver pasaje de argumentos con [*args](https://docs.python.org/3/tutorial/controlflow.html#arbitrary-argument-lists) y [**kwargs](https://docs.python.org/3/tutorial/controlflow.html#keyword-arguments)._

In [ ]:
wrapper(test, 1, 'a', a=3, x=44, n='pepepe')
# Salida esperada:
# Ejecutada test()
# Argumentos posicionales: (1, 'a')
# Argumentos con nombre: {'a': 3, 'x': 44, 'n': 'pepepe'}


### Composición de funciones

Versión imperativa

In [ ]:
def add_elemento(xs: list[int], x: int) -> None:
    xs.append(x)

lista_enteros: list[int] = []
add_elemento(lista_enteros, 1)
add_elemento(lista_enteros, 2)
add_elemento(lista_enteros, 3)
lista_enteros

Versión funcional

In [ ]:
def add_elemento(xs: list[int], x: int) -> list[int]:
    ys: list[int] = xs.copy()
    ys.append(x)
    return ys

lista_enteros: list[int] = add_elemento(add_elemento(add_elemento([], 1), 2), 3)
lista_enteros

## Inmutabilidad

### Transitividad

Recordar que debemos verificar que los atributos de un objeto inmutable sean también inmutables, o de lo contrario contemplar en que no puedan mutar.

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class ContenedorInmutable(Generic[T]):
    def __init__(self, valor: T):
        self._valor: T = valor
    
    def contenido(self) -> T:
        return self._valor

xs: list[int] = [1, 2, 3]
contenedor: ContenedorInmutable[list[int]] = ContenedorInmutable(xs)
xs[0] = 9

print(contenedor.contenido())   # [9, 2, 3]

### Clases inmutables

In [ ]:
class MiClaseInmutable:
    def __init__(self, valor_inicial):
        self._valor = valor_inicial
    
    @property
    def valor(self):
        return self._valor

objeto_inmutable = MiClaseInmutable(20)
objeto_inmutable.valor                      # 20
objeto_inmutable.valor = 10                 # AttributeError: property 'valor' of 'MiClaseInmutable' object has no setter
objeto_inmutable._valor = 10                # Modifica el valor
objeto_inmutable.valor                      # 10

#### Métodos especiales `__setattr__` y `__delattr__`

In [ ]:
class MiClaseInmutable:
    __slots__ = ('_valor',)

    def __init__(self, valor_inicial):
        super().__setattr__('_valor', valor_inicial)
    
    def __setattr__(self, __name: str, __value: Any) -> None:
        raise AttributeError(f'No es posible setear el atributo {__name}')
    
    def __delattr__(self, __name: str) -> None:
        raise AttributeError(f'No es posible eliminar el atributo {__name}')
    
    def valor(self):
        return self._valor

#### Named Tuples

In [ ]:
from collections import namedtuple

MiClaseInmutable = namedtuple('MiClaseInmutable', 'valor1 valor2')
mi_obj = MiClaseInmutable(10, 20)
mi_obj                  # MiClaseInmutable(valor1=10, valor2=20)
mi_obj.valor1           # 10
mi_obj.valor2           # 20

In [ ]:
from collections import namedtuple

class MiClaseInmutable(namedtuple('MiClaseInmutable', 'valor1 valor2')):
    __slots__ = ()
    def __repr__(self) -> str:
        return f'{super().__repr__()} INMUTABLE'
    
mi_obj = MiClaseInmutable(10, 20)
mi_obj                  # MiClaseInmutable(valor1=10, valor2=20) INMUTABLE

#### dataclass

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Persona:
    nombre: str
    apellido: str
    edad: int

    def es_adulta(self):
        return edad >= 18
    
p = Persona("Julia", "Martinez", 22)
print(p)        # Persona(nombre='Julia', apellido='Martinez', edad=22)
p.edad = 20     # FrozenInstanceError: cannot assign to field 'edad'

## Inmutabilidad

### Ejercicio: Conjunto inmutable
Implementar una versión de un conjunto de elementos de cualquier tipo que sea inmutable. Podemos apoyarnos en la `tuple` de Python. El conjunto se crea con una cantidad de elementos variables y luego ya no puede modificarse.

In [ ]:
class ConjuntoInmutable:
    # Implementar
    pass


contenedor = ConjuntoInmutable('a', 1, 3, 'b', 5)
contenedor.elementos        # ('a', 1, 3, 'b', 5)
x.elementos[2] = 10         # Error
x.elementos = (1, 3, 'a')   # Error

## Funciones puras

### Efectos secundarios

In [ ]:
def duplicar_elemento(lista: list[int], indice: int) -> list[int]:
    lista[indice] *= 2
    return lista

def duplicar_elemento_pura(lista: list[int], indice: int) -> list[int]:
    nueva_lista = lista.copy()
    nueva_lista[indice] *= 2
    return nueva_lista

# Uso de ambas funciones
original: list[int] = [1, 2, 3]
resultado1: list[int] = duplicar_elemento(original, 1)
resultado2: list[int] = duplicar_elemento_pura(original, 1)

print(f"Impura: {resultado1}")  # Salida: [1, 4, 3]
print(f"Pura: {resultado2}")  # Salida: [1, 8, 3]

### Ejercicio: Funciones puras e impuras
Proponer ejemplos de funciones impuras para cada tipo de efecto secundario mencionado y cómo se podrían conventir, si es posible, a versiones de funciones puras.

#### Modificación de Variables Globales

#### Modificación de Argumentos

### Impresiones en Consola o Registro de Eventos

## Estrategias de evaluación

### Evaluación estricta

In [ ]:
def func1():
    print('Evalua funcion 1')
    return 1
def func2():
    print('Evalua funcion 2')
    return 2
def sumaRara(x, y):
    print('Evalua funcion externa')
    return x if x == 1 else x + y

sumaRara(func1(), func2())

### Evaluación no estricta

#### Evaluación de cortocircuito

In [ ]:
def esDivisor(nro: int, divisor: int) -> bool:
    return (divisor > 0) and (nro % divisor == 0)

esDivisor(10, 0)    # False

#### Evaluación perezosa

Al finalizar de consumirse el iterador, se produce el StopIteration.

In [ ]:
from collections.abc import Iterator

def genera_saludo() -> Iterator[str]:
    yield "Hola"
    yield "Buenas"
    yield "Buen día"

iterador_saludos = genera_saludo()
print(next(iterador_saludos))   # Hola
print(next(iterador_saludos))   # Buenas
print(next(iterador_saludos))   # Buen día
print(next(iterador_saludos))   # Error StopIteration

Si se consume un iterador, ya no es posible reiniciarlo, se debe generar otro.

In [ ]:
iterador_saludos = genera_saludo()
for saludo in iterador_saludos:
    print(saludo)
for saludo in iterador_saludos:
    print(saludo)

Función generadora de positivos pares.

In [ ]:
from collections.abc import Iterator

def positivos_pares() -> Iterator[int]:
    numero: int = 0
    while True:
        yield numero
        numero += 2

Versión con expresión generadora.

In [ ]:
positivos_pares = (x for x in range(0, 10, 2))  # <generator object <genexpr> ...>
list(positivos_pares)

Expresión generadora que produce funciones.

In [ ]:
funciones = (lambda z: x * 2 + z for x in range(5))
# Los valores se generan solo cuando se solicita.
print(next(funciones)(10))  # Se evalúa el primer valor cuando se solicita.
print(next(funciones)(10))  # Se evalúa el segundo valor cuando se solicita.
print(next(funciones)(20))  # Se evalúa el tercer valor cuando se solicita.
print(next(funciones)(10))  # Se evalúa el cuarto valor cuando se solicita.

### Ejercicio: Generador de primos
Implementar una función generadora que permita producir todos los números primos uno a uno.


In [ ]:
from collections.abc import Iterator

def generador_primos(n: int) -> Iterator[int]:
    # Implementar...
    pass
 
for i in generador_primos(100):
    print(i)

### Ejercicio: Pipeline de datos con generadores
En ciertos casos podemos encontrarnos con archivos CSV muy grandes que no entren en memoria para procesarlos completamente, por lo cual veremos una forma de procesar datos a demanda a medida que se leen. Se pide implementar: 
- un lector de archivo CSV utilizando 3 generadores:
    - uno para producir cada línea leída del archivo.
    - otro para producir una lista de campos _string_ a partir de cada línea leída, consumiendo el generador previo.
    - otro para producir un diccionario a partir de cada lista de campos obtenida con el generador previo.
- calcular la suma de los _sepal\_width_ de todas las especies _Iris-setosa_ del dataset [IRIS.csv](../datasets/IRIS.csv), utilizando un generador que produzca cada valor de _sepal\_width_ de una planta a la vez que sea de esa especie. _Valor esperado: 170.9_
- similar al punto anterior, pero calculando el promedio del _sepal\_width_ de las especies _Iris-setosa_. _Valor esperado: 3.418_

_TIP: Ver la función [`open()`](https://docs.python.org/3/library/functions.html#open) para leer archivos de texto._

## Transformación de funciones


### Currificación

In [ ]:
# Función simple de suma
def suma(x, y):
    return x + y

# Función currificada de suma
def suma_curry(x):
    def suma_x(y):
        return x + y
    return suma_x

print(suma(1, 3))
print(suma_curry(1)(3))

In [ ]:
def doble(x):
    return suma_curry(x)(x)

def incrementar_10(x):
    return suma_curry(10)(x)

print(doble(8))
print(incrementar_10(9))

In [ ]:
def suma_xyz(x):
    def suma_x(y):
        def suma_y(z):
            return x + y + z
        return suma_y
    return suma_x

suma_xyz(1)(2)(3)

#### functools.partial

In [ ]:
from functools import partial

def producto(x: int, y: int) -> int:
    return x * y

producto_10 = partial(producto, 10)
producto_10(2)

#### pymonad.tools.curry

In [ ]:
from pymonad.tools import curry

@curry(2)
def producto(x: int, y: int) -> int:
    return x * y

producto_10 = producto(10)
producto_10(2)

In [ ]:
from pymonad.tools import curry

producto_curry = curry(2, producto)
producto_10 = producto_curry(10)
producto_10(2)

### Ejercicio: Registrando logs
A lo largo de nuestro programa es posible que necesitemos almacenar información de interés en el log de ejecución. A efectos prácticos, nuestro destino de log será la consola, por lo que podemos utilizar simplemente un `print()` para registrar un mensaje de log.

Implementar una función `log` _currificada_ que permita registrar un mensaje de log y el tipo, que puede ser _error_, _alerta_ o _información_.

### Composición con decoradores

In [ ]:
from collections.abc import Callable
from functools import wraps

def trim(f: Callable[[str], str]) -> Callable[[str], str]:
    @wraps(f)
    def wrapper(texto: str) -> str:
        return f(texto).strip()
    return wrapper

@trim
def transforma_texto(texto: str) -> str:
    return texto.replace('.',' ')

transforma_texto('  esto es una prueba. ')  # 'esto es una prueba'

Ahora parametrizado

In [ ]:
from collections.abc import Callable
from functools import wraps

def trim(inicio: bool = True, fin: bool = True) -> Callable[[Callable[[str], str]], Callable[[str], str]]:
    def trim_deco(f: Callable[[str], str]) -> Callable[[str], str]:
        @wraps(f)
        def wrapper(texto: str) -> str:
            texto = f(texto)
            if inicio:
                texto = texto.lstrip()
            if fin:
                texto = texto.rstrip()
            return texto
        return wrapper
    return trim_deco

@trim(inicio=False)
def transforma_texto(texto: str) -> str:
    return texto.replace('.',' ')

transforma_texto('  esto es una prueba. ')  # '  esto es una prueba'

In [ ]:
@trim(fin=False)
def transforma_texto(texto: str) -> str:
    return texto.replace('.',' ')

transforma_texto('  esto es una prueba. ')  # 'esto es una prueba  '

### Ejercicio: Decorando para _valores faltantes_
En ciertas situaciones veremos que una función no siempre puede devolver un valor como esperamos. Dependiendo de los argumentos recibidos, es posible que la función produzca algún error durante su evaluación o simplemente no encuentre un valor apropiado a devolver. En la programación funcional se suele utilizar la mónada _Maybe_ para resolver este problema, pero nosotros itentearemos una solución más sencilla.

Se pide implementar una función decoradora `acepta_no_valor` que permita adaptar una función con un único parámetro de cualquier tipo no nulo de forma que devuelva la evaluación de esa función si el argumento recibido no es `None`. De lo contrario, debe devolver `None`.

TIP: Se puede usar el _hint_ de tipo de retorno de la decoradora como: `Callable[[T | None], R | None]`. Ver [Generics](../A_Python_POO/README.md#generics).

In [ ]:
# Implementar decorador acepta_no_valor

@acepta_no_valor
def incrementar(x: int) -> int:
    return x + 1

incrementar(20)     # 21
incrementar(None)   # None

## Iteraciones e iterables

Estilo imperativo

In [ ]:
def potencia2(n: int) -> int:
    retorno: int = 1
    for x in range(0, n):
        retorno *= 2
    return retorno

potencia2(11)   # 2048

Estilo funcional

In [ ]:
def iterar(veces: int, func: Callable[..., Any], valor: Any) -> Any:
    if veces <= 0:
        return valor
    else:
        return iterar(veces - 1, func, func(valor))
    
def potencia2(n: int) -> int:
    return iterar(n, lambda x: 2 * x, 1)

potencia2(11)   # 2048

### map

In [ ]:
xs: list[int] = [1, 2, 3, 4]
ys: list[int] = []
operacion = lambda x: x * x
for x in xs:
    ys.append(operacion(x))

ys

In [ ]:
cuadrados: map = map(operacion, xs)    # <map at 0x1beb3187940>
list(cuadrados)     # [1, 4, 9, 16]

In [ ]:
totales: list[int] = [100, 200, 300]
registros: list[int] = [50, 40, 120]

proporciones: map = map(lambda x, y: x / y, totales, registros)
list(proporciones)  # [2.0, 5.0, 2.5]

In [ ]:
proporciones: list[float] = [x / y for x, y in zip(totales, registros)]
proporciones

In [ ]:
from collections.abc import Iterable, Iterator
from typing import Any

def mi_zip(*iterables: Iterable[Any]) -> Iterator[tuple[Any, ...]]:
    return map(lambda *elementos: tuple(elementos), *iterables)

list(mi_zip([1,2,3], ['a','b','c']))

### Ejercicio: Contar letras
A través del uso del `map`, dada una lista de cadenas generar una nueva lista que devuelva la cantidad que tiene de cierta letra (debe ser pasada como argumento) cada elemento. 

Por ejemplo, si queremos contar la letra 'a' en ['casa', 'hogar', 'espacio', 'cuento'] deberíamos obtener [2, 1, 1, 0].

In [ ]:
# Implementar contar_a(texto) o contar(letra, texto)

xs = ['casa', 'hogar', 'espacio', 'cuento']
list(map(contar_a, xs))     # [2, 1, 1, 0]

### filter

Estilo imperativo.

In [ ]:
def es_par(n: int) -> bool:
    return n % 2 == 0

xs = [1, 2, 3, 4, 5, 6]
ys = []
for x in xs:
    if es_par(x):
        ys.append(x)

ys

Estilo funcional.

In [ ]:
def es_par(n: int):
    return n % 2 == 0

filter(es_par, [1, 2, 3, 4, 5, 6])  # <filter at 0x1d2af1aed70>
list(filter(es_par, [1, 2, 3, 4, 5, 6]))

Detección de outliers con z-score.

In [ ]:
import numpy as np
from pymonad.tools import curry
from pymonad.reader import Compose

@curry(3)
def zscore(media: float, desvio: float, valor: float) -> float:
    return (valor - media) / desvio

def es_outlier(z_score: float, limite :float = 3) -> bool:
    return z_score > limite or z_score < (limite * -1)

# Generamos muestra random
muestra = np.random.normal(0, 5, 1000)
# Aplicamos parcialmente argumentos a zscore
zscore_muestra = zscore(muestra.mean(), muestra.std())
# Generamos nueva función predicado mediante la composición
filtro_outlier = Compose(zscore_muestra).then(es_outlier)

list(filter(filtro_outlier, muestra))   # lista con outliers

In [ ]:
[ x for x in muestra if es_outlier(zscore_muestra(x)) ]

### reduce

In [ ]:
from functools import reduce

def contar_letras(acumulado: int, elemento: str) -> int:
    return acumulado + len(elemento)

reduce(contar_letras, ['casa', 'puente', 'ojo'], 0)

In [ ]:
from functools import reduce

xs = [3, 4, 1, 0, 11, 7, 5, 6]

# sum(xs)
reduce(lambda x, y: x + y, xs, 0)

In [ ]:
# max(xs)
reduce(lambda x, y: x if x > y else y, xs)

In [ ]:
maximo, *resto = xs
for x in resto:
    if x > maximo:
        maximo = x

maximo

### Ejercicio: Conteo de elementos
Definir utilizando `reduce` una operación que dada una lista de cadenas devuelva un diccionario donde las claves sean cada elemento de la lista y los valores sean la cantidad de apariciones que tiene ese elemento en la lista.

Ejemplo: `contar(['a', 'b', 'c', 'a', 'a', 'c', 'b', 'd', 'c', 'a', 'e'])` -> `{'a': 4, 'b': 2, 'c': 3, 'd': 1, 'e': 1}`.

In [ ]:
from functools import reduce

# Implementar contar(xs: list[str]) -> dict[str, int]

contar(['a', 'b', 'c', 'a', 'a', 'c', 'b', 'd', 'c', 'a', 'e']) # {'a': 4, 'b': 2, 'c': 3, 'd': 1, 'e': 1}

### Ejercicio: Ordenar con reduce
Utilizando la operación `reduce` definir una operación que ordene una lista de números enteros de menor a mayor.

In [ ]:
from functools import reduce

# Implementar ordenar(xs: list[int]) -> list[int]

ordenar([3, 4, 1, 0, 11, 7, 5, 6]) # [0, 1, 3, 4, 5, 6, 7, 11]